# Scrapy Tutorial

공식 문서 튜토리얼 따라하기 [doc.scrapy](https://doc.scrapy.org/en/2.0/intro/tutorial.html)

유명한 인용문들을 [quotes.toscrape.com](http://quotes.toscrape.com/) 에서 스크랩해보자

아래의 순서로 진행,

   1. 새로운 Scrapy project 생성
   2. 사이트를 크롤링하고 데이터를 추출하기 위한 spider 작성
   3. command line을 사용하여 스크랩 된 데이터 내보내기
   4. 재귀적으로 링크를 따라 들어가도록 spider 수정
   5. spider arguments 사용하기


## Creating a project

프로젝트를 생성할 디렉토리를 정하고 아래 명령어를 통해 새로운 프로젝트를 생성한다.

`scrapy startproject <project_name>`

In [2]:
! scrapy startproject tutorial

New Scrapy project 'tutorial', using template directory 'C:\Users\User\anaconda3\envs\web_venv\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\web_scrapping\0. Notebooks\tutorial

You can start your first spider with:
    cd tutorial
    scrapy genspider example example.com


실행하면 __tutorial__ 디렉토리가 생성되고 아래와 같은 구조로 구성되어 있다.

```
tutorial/
    scrapy.cfg            # deploy 설정 파일
    tutorial/             # 프로젝트의 파이썬 모듈, 여기서 코드를 import 한다
        __init__.py
        items.py          # items 정의
        middlewares.py    # middlewares 파일
        pipelines.py      # pipelines 파일
        settings.py       # 설정 파일
        spiders/          # 나중에 나만의 spiders를 넣을 디렉토리
            __init__.py|
```

In [1]:
! tree /F tutorial

폴더 PATH의 목록입니다.
볼륨 일련 번호는 3ADA-D4B8입니다.
C:\USERS\USER\DESKTOP\WEB_SCRAPPING\0. NOTEBOOKS\TUTORIAL
│  scrapy.cfg
│  
└─tutorial
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    ├─spiders
    │  │  __init__.py
    │  │  
    │  └─__pycache__
    └─__pycache__


---
---

## Our first Spider

__Spiders__ 는 웹사이트에서 정보를 스크랩할 때 Scrapy가 사용하는 사용자 정의 클래스들이다. `Spider` 클래스를 상속받아 최초 요청, 선택적으로 페이지의 링크를 따르는 방법, 다운로드한 페이지 내용을 구문 분석하여 데이터를 추출하는 방법 등을 정의해야 한다.

`tutorial/spiders` 디렉토리 아래에 새로운 파일 `quotes_spider.py`를 생성하고 아래의 코드를 입력하여 나의 첫 Spider를 만들어보자.

In [3]:
%%writefile tutorial/tutorial/spiders/quotes_spider.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    
    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'quotes-{page}.html'
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log(f'Saved file {filename}')

Writing tutorial/tutorial/spiders/quotes_spider.py


__scrapy.Spider__ 를 상속받고 속성과 메소드들 몇 개를 추가하였다.

- `name` : 
    - Spider를 구분하기 때문에 프로젝트에서 unique한 값을 가져야 한다.

- `start_requests()` : 
    - Requests 반복자를 반환해야한다. (list로 반환해도 되고 위 코드처럼 generator 함수를 반환해도 된다)

- `parse()` : 
    - 각 요청(request)에 대해 다운로드되는 응답(response)을 처리하는 메소드를 호출
    - response 인자는 페이지 내용을 보관하는 TextResponse의 한 인스턴스
    - 주로 response를 구문 분석하고, dicts 타입으로 스크랩된 데이터를 추출하고 새로운 URLs를 찾아 new requests 폼을 생성한다.

---

### How to run our spider

__프로젝트 가장 상위 디렉토리__ 로 이동하여 아래 명령어를 실행한다.

`scrapy crawl <spider_name>`

In [5]:
% cd tutorial
! scrapy crawl quotes

C:\Users\User\Desktop\web_scrapping\0. Notebooks\tutorial


2020-04-24 12:39:11 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: tutorial)
2020-04-24 12:39:11 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.2 (default, Apr 14 2020, 19:01:40) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-24 12:39:11 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'tutorial', 'NEWSPIDER_MODULE': 'tutorial.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['tutorial.spiders']}
2020-04-24 12:39:11 [scrapy.extensions.telnet] INFO: Telnet Password: 83c7df4d484c55d6
2020-04-24 12:39:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-04-24 12:39:11 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsT

우리가 생성한 `quotes` spider를 통해 [quotes.toscrape.com](http://quotes.toscrape.com/) 도메인에 requests를 보냈다. 결과는 아래와 비슷하게 출력되는 것을 볼 수 있다.

``` java
... (생략)
2016-12-16 21:24:05 [scrapy.core.engine] INFO: Spider opened
2016-12-16 21:24:05 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2016-12-16 21:24:05 [scrapy.extensions.telnet] DEBUG: Telnet console listening on 127.0.0.1:6023
2016-12-16 21:24:05 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://quotes.toscrape.com/robots.txt> (referer: None)
2016-12-16 21:24:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/1/> (referer: None)
2016-12-16 21:24:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/2/> (referer: None)
2016-12-16 21:24:05 [quotes] DEBUG: Saved file quotes-1.html
2016-12-16 21:24:05 [quotes] DEBUG: Saved file quotes-2.html
2016-12-16 21:24:05 [scrapy.core.engine] INFO: Closing spider (finished)
```

현재 디렉토리(`/tutorial`)를 살펴보면 두 개의 새로운 파일 "quotes-1.html", "quotes-2.html" 이 생성된 것을 볼 수 있다.

#### What just happend under the hood?

Scrapy가 스케줄을 짠다. Spider의 `start_requests()` 메서드로 반환된 객체 요청(`scrapy.Request`) 각각에 대한 응답(`Response`)을 받으면, 응답 객체를 인스턴스화하고, 요청과 관련된 콜백 방법(이 경우, `parse` 방법)을 인수로 전달하여 호출한다.

---

### A shortcut to the start_requests method

`start_requests()` 메소드를 구현하는 것 말고 그냥 `start_urls` 클래스 속성을 정의할 수 있다. 그러면 spider에 기본으로 구현된 `start_requests()` 메소드에 사용되어 initial 요청을 생성한다.

In [7]:
%%writefile tutorial/spiders/quotes_spider.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)

Overwriting tutorial/spiders/quotes_spider.py


특별한 명시가 없었는데도 `parse()` 메소드가 리스트에 있는 URLs의 요청을 처리하는 것을 볼 수 있다. 이는 Scrapy의 기본 callback 메소드가 `parse()`이기 때문이다.

---

### Extracting data

Scrapy를 사용한 데이터 추출을 배우는 가장 좋은 방법은 [Scrapy shell](https://doc.scrapy.org/en/2.0/topics/shell.html#topics-shell)을 사용하는 것이다.

> shell이란? 스파이더를 실행하지 않고도 스크래핑 코드를 빠르게 시도하고 디버깅 할 수 있다. 데이터 추출 코드를 테스트 하는 데 사용하지만, 일반 python 셸이므로 모든 종류의 코드를 테스트 가능하다.

shell 환경에서는 `scrapy shell <URL>` 명령어로 간단하게 실행 가능하다.

ex) `scrapy shell "http://quotes.toscrape.com/page/1/"`


여기 노트북에서는 아래와 같은 방법으로 실행한다.

참고) [stackoverflow](https://stackoverflow.com/questions/49908158/using-scrapy-in-jupyter-notebook-accessing-response-directly)

In [2]:
import requests
from scrapy.http import TextResponse


res = requests.get("http://quotes.toscrape.com/page/1/")
response = TextResponse(res.url, body=res.text, encoding='utf-8')

__CSS__ 를 사용하여 response 객체의 특정 elements를 선택할 수 있다.

In [13]:
response.css('title')

[<Selector xpath='descendant-or-self::title' data='<title>Quotes to Scrape</title>'>]

`SelectorList` 라는 list-like 객체를 반환한다.

XML/HTML 요소들로 감싸진 `Selector` 객체들의 리스트를 나타낸다.

나아가 query를 통해 선택 항목을 미세하게 그라인(fine-grine)하거나 데이터를 추출할 수 있다.

<br><br><br>


__title에서 텍스트를 추출__

In [3]:
response.css('title::text').getall()

['Quotes to Scrape']

여기서 주목해야 할 두 가지가 있는데, 

- 첫째는 CSS 쿼리에 `::text`, 즉 `<title>` 요소 바로 안에 있는 텍스트 요소만을 선택하고자 함을 의미한다. 

    `::text`를 지정하지 않으면 태그를 포함한 전체 제목 요소를 얻게 된다.

In [4]:
response.css('title').getall()

['<title>Quotes to Scrape</title>']

- 다른 하나는 `.getall()`을 호출한 결과가 list라는 것이다. 

    첫 번째 결과만 얻고 싶다면 `get()`을 사용하면 된다.

In [5]:
response.css('title::text').get()

'Quotes to Scrape'

`re()` 메소드를 사용한 정규표현식으로도 데이터를 추출할 수 있다.

In [9]:
response.css('title::text').re(r'Quotes.*')

['Quotes to Scrape']

In [7]:
response.css('title::text').re(r'Q\w+')

['Quotes']

In [8]:
response.css('title::text').re(r'(\w+) to (\w+)')

['Quotes', 'Scrape']



#### XPath: a brief intro

[CSS](https://www.w3.org/TR/selectors/) 이외에도 Scrapy의 selector는 [XPath](https://www.w3.org/TR/xpath/all/) 표현식을 지원한다

In [15]:
response.xpath('//title')

[<Selector xpath='//title' data='<title>Quotes to Scrape</title>'>]

In [14]:
response.xpath('//title/text()').get()

'Quotes to Scrape'

__XPath expression__ 은 매우 강력하며 Scrapy Selectors의 기초가 된다. 

실제로는 CSS selector들이 최신형 XPath로 변환되는 것이다.


XPath 표현은 CSS selector들 만큼 인기있지 않지만, 구조를 탐색하는 것 외에도 내용을 볼 수 있기 때문에 유용하다. XPath를 사용하면 다음과 같은 것을 할 수 있다 :

`select the link that contains the text "Next Page"`

#### Extracting quotes and authors

이제 웹페이지에서 인용문을 추출할 코드를 작성해서 spider을 완성하자.

[http://quotes.toscrape.com](http://quotes.toscrape.com) 의 각 인용문은 다음과 같은 HTML 형식으로 표현된다.

``` HTML

<div class="quote">
    <span class="text">“The world as we have created it is a process of our
    thinking. It cannot be changed without changing our thinking.”</span>
    <span>
        by <small class="author">Albert Einstein</small>
        <a href="/author/Albert-Einstein">(about)</a>
    </span>
    <div class="tags">
        Tags:
        <a class="tag" href="/tag/change/page/1/">change</a>
        <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
        <a class="tag" href="/tag/thinking/page/1/">thinking</a>
        <a class="tag" href="/tag/world/page/1/">world</a>
    </div>
</div>

```

<br><br>

__scrapy shell을 사용하여 어떻게 데이터를 추출할지 연습해보자.__

> class가 "quote"인 div 태그를 요청

In [16]:
res = requests.get("http://quotes.toscrape.com")
response = TextResponse(res.url, body=res.text, encoding='utf-8')

In [21]:
response.css("div.quote")

[<Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtype...'>,
 <Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtype...'>,
 <Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtype...'>,
 <Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtype...'>,
 <Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtype...'>,
 <Selector xpath="descendant-or-self::div[@class and contains(concat(' ', normalize-space(@class), ' '), ' quote ')]" data='<div class="quote" itemscope itemtyp

응답받는 많은 인용문 중에서 첫번째 인용문을 변수에 저장하고, 여기서 `text`, `author`, `tags` 를 각각 추출해보자.

In [28]:
quote = response.css("div.quote")[0]

In [30]:
text = quote.css("span.text::text").get()
text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [34]:
author = quote.css("small.author::text").get()
author

'Albert Einstein'

여러개의 태그가 있기 때문에 `getall()` 메소드를 사용한다

In [36]:
tags = quote.css("div.tags a.tag::text").getall()
tags

['change', 'deep-thoughts', 'thinking', 'world']

조금 더 깔끔하게 파이썬 딕셔너리로 저장하여 출력해보자.

> 깔끔하게 보이기 위해 `pprint` 패키지 사용했음

In [38]:
from pprint import pprint

for quote in response.css("div.quote"):
    text = quote.css("span.text::text").get()
    author = quote.css("small.author::text").get()
    tags = quote.css("div.tags a.tag::text").getall()
    
    pprint(dict(text=text, author=author, tags=tags))

{'author': 'Albert Einstein',
 'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
 'text': '“The world as we have created it is a process of our thinking. It '
         'cannot be changed without changing our thinking.”'}
{'author': 'J.K. Rowling',
 'tags': ['abilities', 'choices'],
 'text': '“It is our choices, Harry, that show what we truly are, far more '
         'than our abilities.”'}
{'author': 'Albert Einstein',
 'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles'],
 'text': '“There are only two ways to live your life. One is as though nothing '
         'is a miracle. The other is as though everything is a miracle.”'}
{'author': 'Jane Austen',
 'tags': ['aliteracy', 'books', 'classic', 'humor'],
 'text': '“The person, be it gentleman or lady, who has not pleasure in a good '
         'novel, must be intolerably stupid.”'}
{'author': 'Marilyn Monroe',
 'tags': ['be-yourself', 'inspirational'],
 'text': "“Imperfection is beauty, madness is genius and it's bett

---

### Extracting data in our spider

이제 다시 spider 로 돌아가보자.

현재 spider의 기능은 전체 HTML 페이지를 local 파일로 저장하는 것 뿐이다.

위에서 했던 과정을 spider에 적용시키자.

<br>

Scrapy의 spider는 많은 딕셔너리들을 생성하기 때문에 `yield` 키워드를 사용해 callback하겠다.

In [42]:
%%writefile tutorial/spiders/quotes_spider.py

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

Overwriting tutorial/spiders/quotes_spider.py


파일을 수정하고 spider를 실행하면 아래와 같은 결과를 볼 수 있다.

```
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['life', 'love'], 'author': 'André Gide', 'text': '“It is better to be hated for what you are than to be loved for what you are not.”'}
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['edison', 'failure', 'inspirational', 'paraphrased'], 'author': 'Thomas A. Edison', 'text': "“I have not failed. I've just found 10,000 ways that won't work.”"}
```

In [43]:
! scrapy crawl quotes

2020-04-28 23:43:16 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: tutorial)
2020-04-28 23:43:16 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.2 (default, Apr 14 2020, 19:01:40) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-28 23:43:16 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'tutorial', 'NEWSPIDER_MODULE': 'tutorial.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['tutorial.spiders']}
2020-04-28 23:43:16 [scrapy.extensions.telnet] INFO: Telnet Password: cc4e1872132f57e8
2020-04-28 23:43:16 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-04-28 23:43:16 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsT

---
---

## Storing the scraped data

스크랩한 데이터를 저장하는 가장 간단한 방법은 [Feed exports](https://doc.scrapy.org/en/2.0/topics/feed-exports.html#topics-feed-exports) (JSON, CSV 등)를 사용하는 것이다.

아래 명령어를 통해 JSON 파일을 생성하자.

In [45]:
! scrapy crawl quotes -o quotes.json

2020-04-28 23:49:54 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: tutorial)
2020-04-28 23:49:54 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.2 (default, Apr 14 2020, 19:01:40) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-28 23:49:54 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'tutorial', 'FEED_FORMAT': 'json', 'FEED_URI': 'quotes.json', 'NEWSPIDER_MODULE': 'tutorial.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['tutorial.spiders']}
2020-04-28 23:49:54 [scrapy.extensions.telnet] INFO: Telnet Password: 03df4fb460b8477f
2020-04-28 23:49:54 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-04-28 23:49:54 [scrapy.mi

역사적인(historic) 이유로(?) 같은 명령을 두 번 실행하면 Scrapy는 파일을 덮어씌우는게 아니라 뒤에 덧붙인다.

즉 명령어를 다시 실행하기 전에 이전 파일을 지워야한다.